*Import libraries (tensorflow backend)*

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,ZeroPadding2D,Convolution2D,Dropout,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
from keras.optimizers import SGD
from keras import optimizers
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.inception_v3 import preprocess_input
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.models import Sequential
from keras.layers import Dense, Flatten, AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.resnet50 import preprocess_input


In [ ]:
img_width, img_height = 224, 224
train_data = '../input/chest_xray/chest_xray/train'
test_data = '../input/chest_xray/chest_xray/test'
val_data = '../input/chest_xray/chest_xray/val'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)
val_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    val_data,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical')
input_shape=(224, 224,3)

In [ ]:
# input_shape = (224, 224, 3)
# training_set = train_datagen.flow_from_directory('../input/chest_xray/chest_xray/train',
#                                                  target_size = (224,224),
#                                                  batch_size = 32,
#                                                  class_mode = 'categorical')
# val_set = val_datagen.flow_from_directory('../input/chest_xray/chest_xray/val',
#                                             target_size = (224,224),
#                                             batch_size = 32,
#                                             class_mode = 'categorical')
# test_set = test_datagen.flow_from_directory('../input/chest_xray/chest_xray/test',
#                                             target_size = (224,224),
#                                             batch_size = 32,
#                                             class_mode = 'categorical')
# input_shape = (224, 224, 3)

# MODEL-1:Simple CNN


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation="relu", input_shape=input_shape))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(Dense(activation = 'relu', units = 128))
model.add(Dense(activation = 'sigmoid', units = 2))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

****Convolution**

Choose 32 feature detectors and an input shape of 3D image of 64x64 pixels

***Pooling***

Pooling is made with a 2x2 array 

Add 2nd convolutional layer with the same structure as the 1st to improve predictions

***Flattening***

***Full Connection***

CNN has 128 nodes in the first layer of the ANN that's connected in the backbone with rectifier activation function.  We then add sigmoid activation function because we have binary outcome with 1 node in the output layer.

***Compile the CNN***

adam is for stohastic gradient descent and binary crossentropy for logarithmic loss for binary outcomes

***Image Augmentation***

Apply several transformations to train the model in a better significant way, keras documentation provides all the required information for augmentation

Target size is 64x64. This is the size of the images the model is trained above and size of the batches in which random samples of our images will be included. Class mode is binary because dependent variable is binary.

# MODEL-2:Modified CNN

In [ ]:
#Initialize model
model = Sequential()

#CNN Layer 1
#input: 28X28 images with 1 channel -> (28X28X1) tensors.
#this applies 64 convolution filters of size 3X3 each.
model.add(ZeroPadding2D((1,1), input_shape=input_shape))
model.add(Convolution2D(filters=64, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Dropout(0.4))

#CNN Layer 2
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(filters=64, kernel_size=(3,3), activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.4))

#Flatten the output before feeding these to the fully connected Neurons                                    
model.add(Flatten())

#Fully connected Layers
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid', kernel_initializer='he_normal'))

print(model.summary())
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# MODEL-3:RESNET50

In [ ]:
#import ResNet50 with pre-trained weights. do not include fully #connected layers
base_model = ResNet50(weights='imagenet', include_top=False)
# result = model.output
# result = GlobalAveragePooling2D()(result)
# # add a fully-connected layer
# result = Dense(512, activation='relu')(result)
# # and a fully connected output/classification layer
# predictions = Dense(2, activation='sigmoid')(result)
# model_resnet = Model(inputs=model.input, outputs=predictions)
# base_model=ResNet50(include_top=False, weights='imagenet',input_shape=input_shape)
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
#Fully connected Layers
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))                            
model.compile(loss='categorical_crossentropy',optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])
model.summary()

# MODEL-4:InceptionV3

In [ ]:
from keras.optimizers import Adam
from keras.models import Model
from keras import applications

base_model=applications.InceptionV3(include_top=False, weights='imagenet')
# result = model.output
# result = GlobalAveragePooling2D()(result)
# # add a fully-connected layer
# result = Dense(512, activation='relu')(result)
# # and a fully connected output/classification layer
# predictions = Dense(2, activation='sigmoid')(result)
# model_incep = Model(inputs=model.input, outputs=predictions)
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
#Fully connected Layers
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))                            
model.compile(loss='categorical_crossentropy',optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])
model.summary()

# Training and Validation

In [ ]:
history = model.fit_generator(train_generator,
                              epochs=16,
                              shuffle=True,
                              validation_data=validation_generator, 
                              verbose=1)

# Accuracy and Loss Graphs for training and validation-

In [ ]:
#Accuracy
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training set', 'Test set'], loc='upper left')
plt.show()

#Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training set', 'Test set'], loc='upper left')
plt.show()

# Evaluation-

In [ ]:
scores = model.evaluate_generator(test_generator)
print('acc =',scores[1]*100)

In [ ]:
print(scores[0],scores[1])